In [19]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import my_library.constants as constants

In [20]:
fit_systematics = pd.read_csv('/work/halld/home/viducic/systematic_errors/max_changes.csv')
signifigant_cuts = pd.read_csv('/work/halld/home/viducic/systematic_errors/signifigant_cuts.csv')
kstar_cuts = signifigant_cuts[signifigant_cuts['cut'].str.contains('kstar')]
signifigant_cuts = signifigant_cuts[signifigant_cuts['cut'].isin(['kinfit_cl', 'ppi', 'kp', 'ksp'])]
path_to_fit_result_files = '/work/halld/home/viducic/data/fit_params/{}/tf1_gaus_cross_section_values.csv'
df_pipkmks = pd.read_csv(path_to_fit_result_files.format('pipkmks'))
df_pipkmks['cross_section_percent_error'] = df_pipkmks['cross_section_error']/df_pipkmks['cross_section']
df_pimkpks = pd.read_csv(path_to_fit_result_files.format('pimkpks'))
df_pimkpks['cross_section_percent_error'] = df_pimkpks['cross_section_error']/df_pimkpks['cross_section']


In [21]:
cut_fit_errors = pd.DataFrame(columns=['channel', 'e', 't', 'stat_error', 'fit_error', 'cut_error', 'neutral_kstar_percent_error', 'sys_error', 'total_percent_error', 'cs_error_up', 'cs_error_down'])

In [ ]:
grouped = fit_systematics.groupby(['channel', 'e', 't'])
for (channel, e, t), group in grouped:
    # print(channel, e, t)
    # print(group)
    fit_error = group['max_percent_change'].values[0]
    etc_sig_cuts = signifigant_cuts[(signifigant_cuts['channel'] == channel) & (signifigant_cuts['e'] == e) & (signifigant_cuts['t'] == t)]
    if len(etc_sig_cuts) == 0:
        cut_error = 0
    else:
        etc_sig_cuts['mag_change'] = np.where(True, abs(etc_sig_cuts['percent_change']), 0)
        imax = etc_sig_cuts['mag_change'].idxmax()
        cut_error = etc_sig_cuts.loc[imax]['percent_change']
    try:
        neutral_kstar_percent_error = kstar_cuts[(kstar_cuts['channel'] == channel) & (kstar_cuts['e'] == e) & (kstar_cuts['t'] == t)]['percent_change'].values[0]
    except IndexError:
        neutral_kstar_percent_error = 0
    middle_of_bin = constants.T_CUT_DICT[t][0] + constants.T_WIDTH_DICT[t]/2
    if channel == 'pipkmks':
        stat_error = df_pipkmks[(df_pipkmks['beam_energy'] == e) & (df_pipkmks['t_bin_middle'] == middle_of_bin)]['cross_section_percent_error'].values[0]
        cs = df_pipkmks[(df_pipkmks['beam_energy'] == e) & (df_pipkmks['t_bin_middle'] == middle_of_bin)]['cross_section'].values[0]
    else:
        stat_error = df_pimkpks[(df_pimkpks['beam_energy'] == e) & (df_pimkpks['t_bin_middle'] == middle_of_bin)]['cross_section_percent_error'].values[0]
        cs = df_pimkpks[(df_pimkpks['beam_energy'] == e) & (df_pimkpks['t_bin_middle'] == middle_of_bin)]['cross_section'].values[0]
    sys_error = np.sqrt(fit_error**2 + cut_error**2)
    total_percent_error = np.sqrt(stat_error**2 + sys_error**2)
    cs_error_up = cs*total_percent_error
    cs_error_down = cs*np.sqrt(total_percent_error**2 + neutral_kstar_percent_error**2)
    cut_fit_errors = cut_fit_errors.append({'channel': channel, 'e': e, 't': t, 'stat_error': stat_error, 'fit_error': fit_error, 'cut_error': cut_error, 'neutral_kstar_percent_error': neutral_kstar_percent_error, 'sys_error': sys_error, 'total_percent_error': total_percent_error, 'cs_error_up': cs_error_up, 'cs_error_down': cs_error_down}, ignore_index=True)

In [23]:
# print(cut_fit_errors)
cut_fit_errors.to_csv('/work/halld/home/viducic/systematic_errors/cs_total_error.csv', index=False)